In [1]:
import ct.sm as sm
import cantera as ct
import scipy.constants as C
import matplotlib.pyplot as plt
# import CoolProp.CoolProp as cp
import importlib
# import sm
import numpy as np
# import pint
importlib.reload(sm)

<module 'ct.sm' from 'e:\\Code\\autism\\ss13\\supermattercore\\ct\\sm.py'>

In [2]:
new_he = sm.HeatExchanger(191,0,0)
s = ct.Solution('./ct/xgm.yaml')
nu_N2 = sm.CANISTER_VOLUME*sm.ENG_CANISTER_PRESSURE/8.314/293
cool_V = .77 + 1.47 + 13.37

In [3]:
s.TPX = 300, nu_N2*8.314*300/cool_V, 'N2: 1'
q = ct.Quantity(s, moles=nu_N2/1e3)
q.volume

15.610868591456823

In [4]:
cool_V, nu_N2

(15.61, 3743.5314092517165)

In [32]:
s()


  xgm:

       temperature   300 K
          pressure   5.9815e+05 Pa
           density   6.7178 kg/m^3
  mean mol. weight   28.014 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy            1869.8             52381  J
   internal energy            -87169        -2.442e+06  J
           entropy           -520.71            -14587  J/K
    Gibbs function        1.5808e+05        4.4286e+06  J
 heat capacity c_p            1010.7             28314  J/K
 heat capacity c_v            713.91             20000  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2                 1                 1            1.7754
     [   +3 minor]                 0                 0  

